# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
%matplotlib inline
import gzip
import shutil
from struct import unpack
from collections import namedtuple, Counter, defaultdict
from pathlib import Path
from urllib.request import urlretrieve
from urllib.parse import urljoin
from datetime import timedelta
from time import time

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

In [7]:
sns.set_style('whitegrid')

In [8]:
#Return formatted time
def format_time(t):
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return f'{h: 0>2.0f}:{m:0>2.0f}:{s:0>5.2f}'
    

# Data Paths

In [10]:
data_path = Path('data')
itch_store = str(data_path/'itch.h5')
order_book_store = data_path/'order_book.h5'

In [14]:
FTP_URL = 'ftp://emi.nasdaq.com/ITCH/Nasdaq ITCH/'
SOURCE_FILE = '01302020.NASDAQ_ITCH50.gz'

In [15]:
#Downloads and unzip ITCH data if not yet available
def may_be_download(url):
    if not data_path.exists():
        print("Creating dir")
        data_path.mkdir()
    else:
        print("Dir exists")
    
    filename = data_path / url.split('/')[-1]
    
    if not filename.exists():
        print("Downloading...", url)
        urlretrieve(url, filename)
    else:
        print("File exists")
    
    unzipped = data_path / (filename.stem + ".bin")
    if not unzipped.exists():
        print('Unzipping to ', unzipped)
        with gzip.open(str(filename), 'rb') as f_in:
            with open(unzipped, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    else:
        print('File already unpacked')
    return unzipped

In [16]:
file_name = may_be_download(urljoin(FTP_URL, SOURCE_FILE))
date = file_name.name.split('.')[0]

Dir exists
Downloading... ftp://emi.nasdaq.com/ITCH/Nasdaq ITCH/01302020.NASDAQ_ITCH50.gz


URLError: <urlopen error ftp error: TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None)>